In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mikeio.dfs0 import Dfs0, Dataset
import mikeio.generic

In [ ]:
sModel = 'NBH'
ID = 'EU05'
sDir = r'\\DKCPH1-STOR2.DHI.DK\Projects\11825259_WS\Results\NBH\EU\Calib'

In [ ]:
%%time
sDir_1 = r'{}\{}_2014'.format(sDir,ID)
sDir_2 = r'{}\{}_2015'.format(sDir,ID)
sDir_3 = r'{}\{}_2016'.format(sDir,ID)
sDir_4 = r'{}\{}_2017'.format(sDir,ID)
sDir_5 = r'{}\{}_2018'.format(sDir,ID)
outDir = r'{}\{}'.format(sDir,ID)
if not os.path.exists(outDir): os.makedirs(outDir)

def concat(t1,t2,t3,t4,t5,t):
    ds_1 = Dfs0(t1).read()
    ds_2 = Dfs0(t2).read()
    ds_3 = Dfs0(t3).read()
    ds_4 = Dfs0(t4).read()
    ds_5 = Dfs0(t5).read()

    ds_time = np.concatenate((ds_1.time, ds_2.time[1:], ds_3.time[1:], ds_4.time[1:], ds_5.time[1:]), axis=0)
    #ds_time = np.concatenate((ds_1.time, ds_2.time[1:], ds_3.time[1:]), axis=0)
    ds_time = pd.to_datetime(ds_time)
    ds_time.freq = 'H'

    ds = []
    for i in range (len(ds_1.data)):    
        ds1 = np.concatenate((ds_1.data[i], ds_2.data[i][1:], ds_3.data[i][1:],
                              ds_4.data[i][1:], ds_5.data[i][1:]), axis=0)
        #ds1 = np.concatenate((ds_1.data[i], ds_2.data[i][1:], ds_3.data[i][1:]), axis=0)
        ds.append(ds1)
    
    ds = Dataset(data=ds,time=ds_time,items=ds_1.items)
    dfs = Dfs0()
    dfs.write(filename=t,data=ds,title="concatenated")
    
for file in os.listdir(sDir_1):
    if file.endswith(".dfs0"):
        if file.split('_')[2] == "TS":            
            print(file)
            t1 = '{}\{}'.format(sDir_1,file)
            t2 = '{}\{}'.format(sDir_2,file)
            t3 = '{}\{}'.format(sDir_3,file)
            t4 = '{}\{}'.format(sDir_4,file)
            t5 = '{}\{}'.format(sDir_5,file)
            t = '{}\{}'.format(outDir,file)
            #mikeio.generic.concat(infilenames=[t1,t2,t3],outfilename=t)  # Overwrite with latest
            concat(t1,t2,t3,t4,t5,t)                                      # Use first file